In [134]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import collections
import csv

def get_Content(caseno):
    num = int(caseno % 40)
    os.chdir('/home/jack/2_Reject/html/%s'%str(num))
    with open('%s.html'%caseno) as html:
        html_text = html.read()
        soup = BeautifulSoup(html_text, 'html.parser')
        #print(soup.prettify())
        td = soup.find_all('td')
        #print([x.text for x in td])

        D = collections.OrderedDict()
        A = collections.OrderedDict()
        B = collections.OrderedDict()
        C = collections.OrderedDict()
        L = []
        passlist = ["申請人/商標/標章權人","代理人"]
        alist = []
        blist = []
        clist = []
        
        
        cnt = 0
        loop = 0
        flag = False
        flaga = False
        flagb = False
        flagc = False
        flagaa = False
        flagbb = False
        flagcc = False
        cnta = 1
        cntb = 1
        cntc = 1
        CLASS = -1
        cs = []
        
        for x in td:
            if "代理人" == x.text  or "商標名稱" == x.text :
                flaga = False
            if "商標名稱" == x.text:
                flagb = False
            if "類別" in x.text:
                flagc = False
                
            try:
                if flaga == True:
                    
                    if 'colTitle' in x.attrs['class']:
                        #print("A<-"+x.text)
                        alist.append(x.text)
                        flagaa = False
                    else:
                        #print("\tA->"+str(x.text))
                        flagaa = True

                    if flagaa == True:
                        A["%d申請人/商標/標章權人_"%cnta+alist[-1]] = x.text
                        cnta += 1
                    
                elif flagb == True:
                    
                    if 'colTitle' in x.attrs['class']:
                        #print("B<-"+x.text)
                        blist.append(x.text)
                        flagbb = False
                    else:
                        #print("\tB->"+str(x.text))
                        flagbb = True

                    if flagbb == True:
                        B["%d代理人_"%cntb+blist[-1]] = x.text
                        cntb += 1
                        
                elif flagc == True:

                    if 'colTitle' in x.attrs['class']:
                        #print("C<-"+x.text)
                        clist.append(x.text)
                        flagcc = False
                    else:
                        #print("\tC->"+str(x.text))
                        flagcc = True

                    if flagcc == True:
                        C["%s_類別_"%CLASS+clist[-1]] = x.text
                        cntc += 1
                        
                else:
                    if "類別" in  x.text:
                        #print("<-->"+str(x.text)+"<-->")
                        flagc = True
                        flag = False
                        CLASS = x.text.split(" ")[1].strip("\t\n")
                        #print("CLASS = ",CLASS)
                        cs.append(CLASS)
                        
                    elif "申請人/商標/標章權人" ==  x.text:
                        #print("A<-"+str(x.text))
                        flaga = True
                        flag = False
                    elif "代理人" ==  x.text:
                        #print("B<-"+str(x.text))
                        flagb = True
                        flag = False
                    elif 'colTitle' in x.attrs['class']:
                            #print("<-"+x.text)
                            L.append(x.text)
                            flag = False
                    else:
                        #print("\t->"+str(x.text))
                        flag = True

                    if flag == True:
                        D[L[-1]] = x.text
            except:
                continue
                #print("")
                #print(x.text)
                #print("")
        
        
        index  = html_text.find('<td class="col-xs-3 colTitle">商標樣態／圖樣顏色</td>')
        sub = html_text[index:index+85]
        sub = sub.split("<td>")[-1].split("</td>")
        color = sub[0].strip("\t\n")
        #print(color)
        D['商標樣態／圖樣顏色'] = color
        
        '''
        print(A)
        print(B)
        print(C)
        print(D)
        '''
        
        os.chdir('/home/jack/3_HTML to CSV')
        with open('Rej_main.csv', 'a+', newline='') as csvfile:
            writer = csv.writer(csvfile,delimiter=',')
            #writer.writerow(D.keys())
            writer.writerow(D.values())
            
        with open('申請人.csv', 'a+', newline='') as csvfile:
            writer = csv.writer(csvfile,delimiter=',')
            #writer.writerow(["申請案號","申請人/商標/標章權人_中文名稱","申請人/商標/標章權人_中文地址","申請人/商標/標章權人_英文名稱","申請人/商標/標章權人_國籍"])
            for i in range(1,cnta,4):
                temp = [str(caseno),A["%d申請人/商標/標章權人_中文名稱"%i],A["%d申請人/商標/標章權人_中文地址"%(i+1)],A["%d申請人/商標/標章權人_英文名稱"%(i+2)],A["%d申請人/商標/標章權人_國籍"%(i+3)]]
                writer.writerow(temp)
            
        with open('代理人.csv', 'a+', newline='') as csvfile:
            writer = csv.writer(csvfile,delimiter=',')
            #writer.writerow(["申請案號","代理人_中文名稱","代理人_中文地址"])
            for i in range(1,cntb,2):
                temp = [str(caseno),B["%d代理人_中文名稱"%i],B["%d代理人_中文地址"%(i+1)]]
                writer.writerow(temp)
        
        with open('類別.csv', 'a+', newline='') as csvfile:
            writer = csv.writer(csvfile,delimiter=',')
            #writer.writerow(["申請案號","類別","商品/服務名稱","類似組群"])
            for i in cs:
                temp = [str(caseno),i,C["%s_類別_商品/服務名稱"%i],C["%s_類別_類似組群"%i]]
                writer.writerow(temp)
        
    return D
def goback2():
    os.chdir('/home/jack/2_Reject')
def goback3():
    os.chdir('/home/jack/3_HTML to CSV')

In [132]:
D = get_Content(104043064) #Test
print("=====")
print(D)
print("=====")
for x in D.keys():
    try:
        D[x] = D[x].strip("\n\t ")
    except:
        pass
    print(x+" : ",end = "")
    print(D[x])
print("=====")
'''for x in All:
    D[x] = D[x].strip("\n\t ")
    print(x+" : "+D[x])'''

=====
OrderedDict([('核駁號', 'T0371953'), ('商標種類', '商標'), ('申請案號', '104043064'), ('申請日', '104/07/24'), ('核駁公告日期(卷期)', '105/07/16(43-014)'), ('行政救濟紀錄', ''), ('決定/判決日期', ''), ('決定/判決結果', ''), ('優先權日及首次申請國(地區)', ''), ('展覽會優先權日及展覽會名稱', ''), ('核駁條款', '290101、290103'), ('據以核駁號', '\n'), ('商標名稱', '屋聯網'), ('商標樣態／圖樣顏色', '平面/墨色'), ('被核駁圖形', ''), ('被核駁圖樣中文', '屋聯網'), ('被核駁圖樣英文', ''), ('被核駁圖樣日文', ''), ('被核駁圖樣記號', ''), ('商標圖樣描述', ''), ('說明文字內容', '')])
=====
核駁號 : T0371953
商標種類 : 商標
申請案號 : 104043064
申請日 : 104/07/24
核駁公告日期(卷期) : 105/07/16(43-014)
行政救濟紀錄 : 
決定/判決日期 : 
決定/判決結果 : 
優先權日及首次申請國(地區) : 
展覽會優先權日及展覽會名稱 : 
核駁條款 : 290101、290103
據以核駁號 : 
商標名稱 : 屋聯網
商標樣態／圖樣顏色 : 平面/墨色
被核駁圖形 : 
被核駁圖樣中文 : 屋聯網
被核駁圖樣英文 : 
被核駁圖樣日文 : 
被核駁圖樣記號 : 
商標圖樣描述 : 
說明文字內容 : 
=====


'for x in All:\n    D[x] = D[x].strip("\n\t ")\n    print(x+" : "+D[x])'

In [61]:
new = pd.DataFrame([D])
x = x.append(new , ignore_index=True)
os.chdir('/home/jack/3_HTML to CSV')
x.to_csv("check.csv")

In [136]:
def runhtmltocsv(failname):
    
    cnt = 0
    dircnt = 0
    fail = []
    #DF = pd.DataFrame()

    os.chdir('/home/jack/2_Reject/html')
    dirs = os.listdir()
    
    for j in dirs:
        os.chdir('/home/jack/2_Reject/html/%s'%str(j))
        files = os.listdir()
        dircnt += 1
       
        for i in files:
            if '.html' not in i:
                continue
            else:
                name = i.split(".")[0]
                #print(name)
                try:
                    if len(fail) > 30:
                    #if cnt == 5:
                        break
                    D = get_Content(int(name))
                    #new = pd.DataFrame([D])
                    #DF = DF.append(new,ignore_index=True,sort=True)
                    cnt += 1
                except:
                    fail.append(name)
                    print(name)
        goback3()
        with open('CSV Progress2.txt','a+') as t:
            t.write("CSV Progress2: "+str(dircnt)+" / 40; "+str(cnt)+" / 80597, Failed: "+str(len(fail))+"\n")
        print("CSV Progress2: "+str(dircnt)+" / 40; "+str(cnt)+" / 80597, Failed: "+str(len(fail))+"\n")
        
        if len(fail) > 30:
        #if cnt == 5:
            break 
        
    goback3()
    #DF.to_csv("RejectContent.csv")
    if len(fail) > 0:
        with open('%s.csv'%failname,'a+' ,newline='') as fd:
            writer = csv.writer(fd)
            writer.writerow(['caseno'])
            for k in fail:
                writer.writerow([k])
        with open('CSV Progress2.txt','a+') as t:
            for i in fail:
                t.write("Fail: "+str(i)+"\n")
                

In [3]:
All = ['核駁號', '商標種類', '申請案號', '申請日', '核駁公告日期(卷期)', '行政救濟紀錄', '決定/判決日期', '決定/判決結果', '優先權日及首次申請國(地區)', '展覽會優先權日及展覽會名稱', '核駁條款', '據以核駁號', '申請人/商標/標章權人_中文名稱', '申請人/商標/標章權人_中文地址', '申請人/商標/標章權人_英文名稱', '申請人/商標/標章權人_國籍', '代理人_中文名稱', '代理人_中文地址', '商標名稱', '商標樣態／圖樣顏色', '被核駁圖形', '被核駁圖樣中文', '被核駁圖樣英文', '被核駁圖樣日文', '被核駁圖樣記號', '商標圖樣描述', '說明文字內容', '類別', '商品/服務名稱', '類似組群']
All

['核駁號',
 '商標種類',
 '申請案號',
 '申請日',
 '核駁公告日期(卷期)',
 '行政救濟紀錄',
 '決定/判決日期',
 '決定/判決結果',
 '優先權日及首次申請國(地區)',
 '展覽會優先權日及展覽會名稱',
 '核駁條款',
 '據以核駁號',
 '申請人/商標/標章權人_中文名稱',
 '申請人/商標/標章權人_中文地址',
 '申請人/商標/標章權人_英文名稱',
 '申請人/商標/標章權人_國籍',
 '代理人_中文名稱',
 '代理人_中文地址',
 '商標名稱',
 '商標樣態／圖樣顏色',
 '被核駁圖形',
 '被核駁圖樣中文',
 '被核駁圖樣英文',
 '被核駁圖樣日文',
 '被核駁圖樣記號',
 '商標圖樣描述',
 '說明文字內容',
 '類別',
 '商品/服務名稱',
 '類似組群']

In [135]:
runhtmltocsv("Fail")

CSV Progress2: 1 / 40; 1988 / 80597, Failed: 0

CSV Progress2: 2 / 40; 3973 / 80597, Failed: 0

CSV Progress2: 3 / 40; 5993 / 80597, Failed: 0

CSV Progress2: 4 / 40; 8026 / 80597, Failed: 0

CSV Progress2: 5 / 40; 10053 / 80597, Failed: 0

CSV Progress2: 6 / 40; 12097 / 80597, Failed: 0

CSV Progress2: 7 / 40; 14170 / 80597, Failed: 0

CSV Progress2: 8 / 40; 16190 / 80597, Failed: 0

CSV Progress2: 9 / 40; 18175 / 80597, Failed: 0

107071791
CSV Progress2: 10 / 40; 20139 / 80597, Failed: 1

CSV Progress2: 11 / 40; 22118 / 80597, Failed: 1

CSV Progress2: 12 / 40; 24185 / 80597, Failed: 1

CSV Progress2: 13 / 40; 26221 / 80597, Failed: 1

CSV Progress2: 14 / 40; 28197 / 80597, Failed: 1

CSV Progress2: 15 / 40; 30213 / 80597, Failed: 1

CSV Progress2: 16 / 40; 32304 / 80597, Failed: 1

CSV Progress2: 17 / 40; 34293 / 80597, Failed: 1

CSV Progress2: 18 / 40; 36302 / 80597, Failed: 1

CSV Progress2: 19 / 40; 38324 / 80597, Failed: 1

CSV Progress2: 20 / 40; 40400 / 80597, Failed: 1

CSV

人工檢查約10多筆補上漏值
最後有一筆奇怪的東西沒有進CSV(但有可能有檔案)

In [17]:
os.chdir('/home/jack/3_HTML to CSV')
df = pd.read_csv('RejectContent.csv')  
suc = df["申請案號"].to_list()
allrej = []
os.chdir('/home/jack/2_Reject/html')
dirs = os.listdir()

for j in dirs:
    os.chdir('/home/jack/2_Reject/html/%s'%str(j))
    files = os.listdir()
    allrej = allrej + files
    
for x in range(len(allrej)):
    try:
        allrej[x] = int(allrej[x].split(".")[0])
    except:
        allrej[x] = 66666666666
print(len(suc))
print(len(allrej))
print(set(allrej)-set(suc))

80582
80588
{106062528, 99042144, 99029477, 66666666666, 99029244, 99003966}
